> 文章来源:https://zhuanlan.zhihu.com/p/279758736

PyTorch是动态图，即计算图的搭建和运算是同时的，随时可以输出结果；而TensorFlow是静态图。

## pytorch计算图
在pytorch的计算图里只有两种元素：数据（tensor）和 运算（operation）

1. 运算包括了：加减乘除、开方、幂指对、三角函数等可求导运算

2. 数据可分为：叶子节点（leaf node）和非叶子节点；叶子节点是用户创建的节点，不依赖其它节点；它们表现出来的区别在于反向传播结束之后，非叶子节点的梯度会被释放掉，只保留叶子节点的梯度，这样就节省了内存。如果想要保留非叶子节点的梯度，可以使用retain_grad()方法。

torch.tensor 具有如下属性：

1. 查看 是否可以求导 requires_grad
2. 查看 运算名称 grad_fn
3. 查看 是否为叶子节点 is_leaf
4. 查看 导数值 grad

> 针对requires_grad属性，自己定义的叶子节点默认为False，而非叶子节点默认为True，神经网络中的权重默认为True。判断哪些节点是True/False的一个原则就是从你需要求导的叶子节点到loss节点之间是一条可求导的通路。

当我们想要对某个Tensor变量求梯度时，需要先指定requires_grad属性为True，指定方式主要有两种：

In [1]:
import torch

In [2]:
x = torch.tensor(1.).requires_grad_() # 第一种

x = torch.tensor(1., requires_grad=True) # 第二种

PyTorch提供两种求梯度的方法：backward() 和 torch.autograd.grad() ，他们的区别在于前者是给叶子节点填充.grad字段，而后者是直接返回梯度给你，我会在后面举例说明。还需要知道y.backward()其实等同于torch.autograd.backward(y)

# 案例
## 案例1
> 一个简单的求导例子是： $ y = (x + 1) * (x +2)$，计算 $\partial y / \partial x$ ，假设给定 $x = 2$, 先画出计算图


![](https://files.mdnice.com/user/7098/de111e14-c839-4f16-b0e3-870480705a57.png)



手算的话: 
$
\frac {\partial y} {\partial x} = \frac {\partial y} {\partial a}\frac {\partial a} {\partial x} + \frac {\partial y} {\partial b}\frac {\partial b} {\partial x} = (x + 2) * 1 + (x + 1) * 1 \rightarrow 7
$

### 使用backward()

In [3]:
x = torch.tensor(2., requires_grad=True)

a = torch.add(x, 1)
b = torch.add(x, 2)
y = torch.mul(a, b)

y.backward()
print(x.grad)
# >>>tensor(7.)

tensor(7.)



看一下这几个tensor的属性

In [4]:
print("requires_grad: ", x.requires_grad, a.requires_grad, b.requires_grad, y.requires_grad)
print("is_leaf: ", x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
print("grad: ", x.grad, a.grad, b.grad, y.grad)

# >>>requires_grad:  True True True True
# >>>is_leaf:  True False False False
# >>>grad:  tensor(7.) None None None

requires_grad:  True True True True
is_leaf:  True False False False
grad:  tensor(7.) None None None


C:\Users\yuanz\AppData\Local\Temp/ipykernel_9372/335890420.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:485.)
  print("grad: ", x.grad, a.grad, b.grad, y.grad)


使用backward()函数反向传播计算tensor的梯度时，并不计算所有tensor的梯度，而是只计算满足这几个条件的tensor的梯度：
1. 类型为叶子节点、
2. requires_grad=True、
3. 依赖该tensor的所有tensor的requires_grad=True。

所有满足条件的变量梯度会自动保存到对应的grad属性里。

### 使用autograd.grad()

In [5]:
x = torch.tensor(2., requires_grad=True)

a = torch.add(x, 1)
b = torch.add(x, 2)
y = torch.mul(a, b)

grad = torch.autograd.grad(outputs=y, inputs=x)
print(grad[0])
# >>>tensor(7.)

tensor(7.)


因为制定了输入y,输入了x, 所以返回值就是$\partial y / \partial x$ 这一梯度，完整的返回值其实是一个元组，保留第一个元素就行，后面的元素可以不用管。

## 案例2

> 再举一个复杂一点且高阶求导的例子： $z = x^2 y$,计算$\partial z / \partial x, \partial z /\partial y, \partial^2z / \partial ^2x$,假设给定$x = 2, y = 3$

手算的话:$\partial z / \partial x = 2xy = 12, \partial z /\partial y = x^2 = 4, \partial^2z / \partial ^2x = 2y = 6$

### 求一阶导可以用backward()


In [6]:
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)

z = x * x * y

z.backward()
print(x.grad, y.grad)
# >>>tensor(12.) tensor(4.)

tensor(12.) tensor(4.)


### 也可以用autograd.grad()

In [7]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x)
print(grad_x[0])
# >>>tensor(12.)

tensor(12.)


为什么不在这里面同时也求对y的导数呢？

因为无论是backward还是autograd.grad在计算一次梯度后图就被释放了，如果想要保留，需要添加retain_graph=True

In [8]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, retain_graph=True)
grad_y = torch.autograd.grad(outputs=z, inputs=y)

print(grad_x[0], grad_y[0])
# >>>tensor(12.) tensor(4.) 

tensor(12.) tensor(4.)


再来看如何求高阶导，理论上其实是上面的grad_x再对x求梯度，试一下看

In [9]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, retain_graph=True)
grad_xx = torch.autograd.grad(outputs=grad_x, inputs=x)

print(grad_xx[0])
# >>>RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn


报错了，虽然retain_graph=True保留了计算图和中间变量梯度， 但没有保存grad_x的运算方式，需要使用creat_graph=True在保留原图的基础上再建立额外的求导计算图，也就是会把$\partial z / \partial x$这样的运算存下来

In [10]:
# autograd.grad() + autograd.grad()
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, create_graph=True)
grad_xx = torch.autograd.grad(outputs=grad_x, inputs=x)

print(grad_xx[0])
# >>>tensor(6.)

tensor(6.)


grad_xx这里也可以直接用backward()，相当于直接从$\partial z / \partial x = 2xy$开始回传

In [11]:
# autograd.grad() + backward()
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad = torch.autograd.grad(outputs=z, inputs=x, create_graph=True)
grad[0].backward()

print(x.grad)
# >>>tensor(6.)

tensor(6.)


也可以先用backward()然后对x.grad这个一阶导继续求导

In [12]:
# backward() + autograd.grad()
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True)
grad_xx = torch.autograd.grad(outputs=x.grad, inputs=x)

print(grad_xx[0])
# >>>tensor(6.)

tensor(6.)


c:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\autograd\engine.cpp:1064.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



那是不是也可以直接用两次backward()呢？第二次直接x.grad从开始回传，我们试一下

In [13]:
# backward() + backward()
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True) # x.grad = 12
x.grad.backward()

print(x.grad)
# >>>tensor(18., grad_fn=<CopyBackwards>)

tensor(18., grad_fn=<CopyBackwards>)


发现了问题，结果不是6，而是18，发现第一次回传时输出x梯度是12。这是因为PyTorch使用backward()时默认会累加梯度，需要手动把前一次的梯度清零

In [14]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True)
x.grad.data.zero_()
x.grad.backward()

print(x.grad)
# >>>tensor(6., grad_fn=<CopyBackwards>)

tensor(6., grad_fn=<CopyBackwards>)


## 案例3 

有没有发现前面都是对标量求导，如果不是标量会怎么样呢？

In [15]:
x = torch.tensor([1., 2.]).requires_grad_()
y = x + 1

y.backward()
print(x.grad)
# >>>RuntimeError: grad can be implicitly created only for scalar outputs

RuntimeError: grad can be implicitly created only for scalar outputs

报错了，因为只能标量对标量，标量对向量求梯度，x可以是标量或者向量，但y只能是标量；所以只需要先将y转变为标量，对分别求导没影响的就是求和。
此时$x = | x_1, x_2|, y = | {x_1^2},{x_2^2}|, z = y.sum() = {x_1^2} + {x_2^2},$

$\partial z / \partial x_1 = 2 x_1 = 2, \partial z / \partial x_2 = 2 x_2 = 4$

In [16]:
x = torch.tensor([1., 2.]).requires_grad_()
y = x * x

y.sum().backward()
print(x.grad)
# >>>tensor([2., 4.])

tensor([2., 4.])


再具体一点来解释，让我们写出求导计算的雅可比矩阵，$y = [y_1, y_2]$是一个向量，
$J = [\partial y / \partial x_1, \partial y / \partial x_2] = 
\begin{bmatrix}
\frac {\partial y_1} {\partial x_1} & \frac {\partial y_1} {\partial x_2}  \\
\frac {\partial y_2} {\partial x_1} & \frac {\partial y_2} {\partial x_21}  \\
\end{bmatrix}
$,但是我们希望最终求导的结果是$[\partial y / \partial x_1, \partial y / \partial x_2]$,那么怎么得到呢？

注意$\frac {\partial y_1} {\partial x_2}  $ 和$\frac {\partial y_2} {\partial x_1}  $ = 0,也就是说
$
[\partial y / \partial x_1, \partial y / \partial x_2 ] = [1,1]\begin{bmatrix}
\frac {\partial y_1} {\partial x_1} & \frac {\partial y_1} {\partial x_2}  \\
\frac {\partial y_2} {\partial x_1} & \frac {\partial y_2} {\partial x_21}  \\
\end{bmatrix}
$

所以不用y.sum()的另一种方式是

In [17]:
x = torch.tensor([1., 2.]).requires_grad_()
y = x * x

y.backward(torch.ones_like(y))
print(x.grad)
# >>>tensor([2., 4.])

tensor([2., 4.])


也可以使用autograd。上面和这里的torch.ones_like(y) 位置指的就是雅可比矩阵左乘的那个向量。

In [18]:
x = torch.tensor([1., 2.]).requires_grad_()
y = x * x

grad_x = torch.autograd.grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y))
print(grad_x[0])
# >>>tensor([2., 4.])

tensor([2., 4.])


或者

In [19]:
x = torch.tensor([1., 2.]).requires_grad_()
y = x * x

grad_x = torch.autograd.grad(outputs=y.sum(), inputs=x)
print(grad_x[0])
# >>>tensor([2., 4.])

tensor([2., 4.])


# 重点
下面是着重强调以及引申的几点：

## 梯度清零
Pytorch 的自动求导梯度不会自动清零，会累积，所以一次反向传播后需要手动清零

```
x.grad.zero_
```
而在神经网络中，我们只需要执行
```
optimizer.zero_grad()
```
## 使用detach()切断
不会再往后计算梯度，假设有模型A和模型B，我们需要将A的输出作为B的输入，但训练时我们只训练模型B，那么可以这样做：
```
input_B = output_A.detach()
```
如果还是以前面的为例子，将a切断，将只有b一条通路，且a变为叶子节点。
```
x = torch.tensor([2.], requires_grad=True)

a = torch.add(x, 1).detach()
b = torch.add(x, 2)
y = torch.mul(a, b)

y.backward()

print("requires_grad: ", x.requires_grad, a.requires_grad, b.requires_grad, y.requires_grad)
print("is_leaf: ", x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
print("grad: ", x.grad, a.grad, b.grad, y.grad)

>>>requires_grad:  True False True True
>>>is_leaf:  True True False False
>>>grad:  tensor([3.]) None None None
```
## 原位操作 in-place
（改变值，不改变对象地址）

> 叶子节点不可执行 in-place 操作，因为反向传播时会访问原来的对象地址。


# 总结
本篇文章其实不难，但是很多人其实会有点糊涂。主要是两点：
1. 还记得大学学的链式法则么？如果记得，并不难。
2. 了解一下pytorch的技巧即可。